### Assessing Houses Value

In this project we will be using Housing data - City of Ames, Iowa.
The data set describe the sale of individual residential property in Ames, Iowa from 2006 to 2010. The data set contains 2930 observations and a large number of explanatory variables (23 nominal, 23 ordinal, 14 discrete, and 20 continuous) involved in assessing home values.

In [23]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#settings
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_columns = 100

We shall import the data and do some exploratory analysis.

In [2]:
#import data
data = pd.read_table("AmesHousing.tsv")
data.shape
data.head()

(2930, 82)

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


Lets look for missing values in the data.

In [3]:
#missing value columns info
na_col_sum = data.isnull().sum()[data.isnull().sum() > 0]
na_col_dtype = data[data.isnull().sum()[data.isnull().sum() > 0].index].dtypes
              
na_df = pd.DataFrame({"Total_NA":na_col_sum, "Col_Dtype":na_col_dtype})
print("Number of columns with NA values:", len(na_df))
na_df.T

Number of columns with NA values: 27


,Lot Frontage,Alley,Mas Vnr Type,Mas Vnr Area,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Electrical,Bsmt Full Bath,Bsmt Half Bath,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Pool QC,Fence,Misc Feature
Total_NA,490,2732,23,23,80,80,83,80,1,81,1,1,1,1,2,2,1422,157,159,159,1,1,159,159,2917,2358,2824
Col_Dtype,float64,object,object,float64,object,object,object,object,float64,object,float64,float64,float64,object,float64,float64,object,object,float64,object,float64,float64,object,object,object,object,object


We have a total of 27 columns with missing values ranging from 1 to 2917. We shall take below approach to deal with them.
- Drop columns with more than 25% missing values.
- Impute quantitative data with column mean (we will round mean to take care of **year** column as well).
- Impute categorical data with a new category "**missing**".

In [21]:
#let's define few functions

#function for transforming features
def transform_features(input_data):
    #find threshhold value for na's (25%)
    threshold_val = round(len(input_data)/4)
    
    #drop columns with more than threshold_val
    data_aft_drpcols = input_data.dropna(axis=1, thresh= len(data) - threshold_val)
    
    #impute numerical data with column mean
    float_cols = data_aft_drpcols.select_dtypes(include = ['float64'])
    float_cols = float_cols.fillna(round(float_cols.mean()))
    
    data_aft_drpcols = data_aft_drpcols.drop(float_cols.columns, axis=1)
    clean_data_temp = pd.concat([data_aft_drpcols, float_cols], axis=1)
    
    #impute categorical data
    text_cols = clean_data_temp.select_dtypes(include = ['object'])
    text_cols = text_cols.fillna("Missing")
    
    clean_data_temp = clean_data_temp.drop(text_cols.columns, axis=1)
    clean_data = pd.concat([text_cols, clean_data_temp], axis=1) 
    
    return(clean_data)

#function for selecting features
def select_features(input_data):
    return(input_data[["Gr Liv Area", "SalePrice"]])


#function to check null values    
def null_columns(data):
    cols = data.isnull().sum()[data.isnull().sum() > 0]
    tot_cols = len(cols)
    return(tot_cols)


#function to train a model
def train_and_test(input_data):
    #split into train and test sets
    train = input_data[0:1460]
    test = input_data[1460:]
    
    #train model
    train_set = select_features(train)
    train_x = train_set.loc[:, train_set.columns != 'SalePrice']
    train_y = train_set['SalePrice']
    
    test_set = select_features(test)
    test_x = test_set.loc[:, test_set.columns != 'SalePrice']
    test_y = test_set['SalePrice']
    
    lr = LinearRegression()
    lr.fit(train_x, train_y)
    
    predicted_y = lr.predict(test_x)
    
    rmse = np.sqrt(mean_squared_error(test_y, predicted_y))
    return(rmse)

In [8]:
clean_data = transform_features(data)

#print null values
print("Number of columns with null values:", null_columns(clean_data))
print("Shape of clean data:", clean_data.shape)

Number of columns with null values: 0
Shape of clean data: (2930, 77)


In [24]:
train_and_test(clean_data)

57088.25161263909

*to be continued...*